In [1]:
import pandas as pd
import numpy as np

### Import data

In [2]:
# maybe use np.where to reduce the data imported

Read

In [3]:
code = pd.read_csv('data/coding7128.tsv', sep='\t')

In [4]:
clinical = pd.read_csv('data/tpp_gp_clinical_subset.csv')

In [5]:
ic10 = pd.read_csv('data/ic10_lookup.csv')

In [6]:
GPic10 = pd.read_csv('data/GP_ic10_lookup.csv')

In [7]:
scripts = pd.read_csv('data/sample_covid19_tpp_gp_scripts.txt')

have a look at the data

In [8]:
code.head()
# CTV3 codes (used by GPs, in dataset clinical) and the corresponding disease and incidences
# code can also be measurements, but will not be used by us

,coding,meaning
0,-1,redacted - potentially sensitive or identifying
1,-2,redacted - rare occupation
2,.....,Read thesaurus
3,.0...,See 0....
4,.01..,See 01...


In [9]:
clinical
# GP dataset from every patient
# each line corresponds to one incident with a date (event_dt)
# eid: patient number 
# code_type: ?
# value: measured value of a test or whatever

,Unnamed: 0,eid,event_dt,code_type,code,value
0,101084316,4392480,31/12/2020,0,XaERu,11.00
1,9359934,1311969,31/12/2020,0,XaN3z,0.80
2,9360007,1311969,31/12/2020,0,44P6.,0.58
3,111269383,4731119,31/12/2020,0,XaIm8,0.00
4,9360002,1311969,31/12/2020,0,44P5.,1.30
...,...,...,...,...,...,...
49995,46497795,2564395,31/12/2019,0,XaFqt,0.00
49996,46497794,2564395,31/12/2019,0,XaIVg,0.00
49997,41639202,2400982,31/12/2019,0,XaEJK,0.00
49998,116371391,4900832,31/12/2019,0,X1004,0.00


In [10]:
ic10
# description of ICD20 codes

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17931,17931,U84.8,U848,DEFAULT,3.0,Resistance to other specified antimicrobial drug,,,,,,,
17932,17932,U84.9,U849,DEFAULT,3.0,Resistance to unspecified antimicrobial drugs,,,,,,,
17933,17933,U85,U85X,DEFAULT,3.0,Resistance to antineoplastic drugs,,,,,,,
17934,17934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
GPic10
#translates CTV3 codes (read_code) to ICD10 (icd10_code)

,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
0,0,123..,Z831,D,C,M,0.0,0.0
1,1,123..,Z830,A,C,M,0.0,0.0
2,2,1231.,Z831,G,C,M,0.0,0.0
3,3,1232.,Z831,G,C,M,0.0,0.0
4,4,1233.,Z831,G,C,M,0.0,0.0
...,...,...,...,...,...,...,...,...
116372,116372,ZVy..,Z769,D,C,C,0.0,0.0
116373,116373,ZVz..,Z769,D,C,C,0.0,0.0
116374,116374,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116375,116375,"Contains information from NHS Digital, licence...",NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
scripts
# all medication patients got from their GP, incl. the COVID-19 vaccine 
# needs to be complete if want to compare vaccinated vs. non-vaccinated, but seems like not all vaccinations registered here but just the ones done by the GP

,Unnamed: 0,eid,issue_date,dmd_code
0,39278003,3743492,2021-03-22,320031002
1,35914327,3508920,2021-03-15,3591211000001105
2,49790799,4472075,2021-05-03,33405511000001102
3,31447155,3192040,2021-03-15,321745007
4,69964306,5879124,2021-01-19,320031002
...,...,...,...,...
99995,62996879,5393814,2021-01-27,320030001
99996,47834233,4333940,2021-01-03,38893411000001105
99997,33831040,3365447,2021-03-31,318052005
99998,12832421,1889162,2021-03-30,321152004


In [13]:
# get few individuals from large dataset -> may try to call only part of scripts dataset, where code == one of covid vaccine

# try only to get lines from scripts dataset where dmd_code== a vaccine 
# use function read_csv. only where irgendwas

print('lines in samplescripts: ', len(scripts))
print('number of subjects in samplescripts: ', len(scripts['eid'].value_counts()))

lines in samplescripts:  100000
number of subjects in samplescripts:  51104


### Preparing the data

#### Adding column with IC10 categories to clinical dataset for simpler analysis later on

In [14]:
# create a reduced dataset for merging with the clinical dataset 
GPic10_red =  GPic10[['read_code', 'icd10_code']]
# rename to merge ont he ctv3 column with the clinical dataset
GPic10_red = GPic10_red.rename(columns = {'read_code':'code'})
GPic10_red.head()

,code,icd10_code
0,123..,Z831
1,123..,Z830
2,1231.,Z831
3,1232.,Z831
4,1233.,Z831


In [15]:
clinical_ic10 = pd.merge(clinical, GPic10_1, how='inner', on='code')
clinical_ic10

NameError: name 'GPic10_1' is not defined

#### Data Cleaning

In [60]:
# drop unnecessary columns
clinical_ic10.drop(columns = ['code_type', 'value'], inplace = True)
clinical_ic10

,Unnamed: 0,eid,event_dt,code,icd10_code
0,36821530,2238884,31/12/2020,Xabdb,R53X
1,36821530,2238884,31/12/2020,Xabdb,R54X
2,58552832,2968142,31/12/2020,Xabdb,R53X
3,58552832,2968142,31/12/2020,Xabdb,R54X
4,24993994,1832436,31/12/2020,Xabdb,R53X
...,...,...,...,...,...
4545,65073848,3185031,31/12/2019,F4201,E123D
4546,65073848,3185031,31/12/2019,F4201,E133D
4547,65073848,3185031,31/12/2019,F4201,O243D
4548,65073848,3185031,31/12/2019,F4201,O244D


In [61]:
pd.isnull(clinical_ic10).sum()
# no NaN

Unnamed: 0    0
eid           0
event_dt      0
code          0
icd10_code    0
dtype: int64

In [71]:
pd.isnull(ic10).sum()

Unnamed: 0          0
ICD10_CODE          1
ALT_CODE            2
USAGE               2
USAGE_UK            2
DESCRIPTION         2
MODIFIER_4          2
MODIFIER_5          2
QUALIFIERS          2
GENDER_MASK         2
MIN_AGE             2
MAX_AGE             2
TREE_DESCRIPTION    2
dtype: int64

In [72]:
# remove duplicate entries in clinical dataset 
# (for other datasets, it should not be a problem)
clinical_ic10.drop_duplicates(inplace = True) 

# remove NaN in selected column
ic10.dropna(subset=['DESCRIPTION'], inplace = True)

In [73]:
pd.isnull(ic10).sum()

Unnamed: 0          0
ICD10_CODE          0
ALT_CODE            0
USAGE               0
USAGE_UK            0
DESCRIPTION         0
MODIFIER_4          0
MODIFIER_5          0
QUALIFIERS          0
GENDER_MASK         0
MIN_AGE             0
MAX_AGE             0
TREE_DESCRIPTION    0
dtype: int64

In [64]:
# remove not considered clinical events in the clinical dataset (e.g. external factors)

""""
Monica's suggestions: 
remove O-Z (pregnancy, congenital, others, COVID-19, external causes, factors)
maybe keep Q (congenital) as underlying disease, keep all others
"""

'"\nMonica\'s suggestions: \nremove O-Z (pregnancy, congenital, others, COVID-19, external causes, factors)\nmaybe keep Q (congenital) as underlying disease, keep all others\n'

### Find who is vaccinated

The meaning of the SNOMED codes ('dmd_code' in the 'scripts' dataset, where medications that the patients got are listed) were found on: https://services.nhsbsa.nhs.uk/dmd-browser/amp/view/159421?ref=YW1wTmFtZT1jb3ZpZCZzZWFyY2hUeXBlPUFNUCZzaG93SW52YWxpZEl0ZW1zPWZhbHNl#ampCodes

Each vaccine has a specific code.

In [18]:
# broadcasting to find individuals with AstraZEneca vaccine
mask_AZ = scripts['dmd_code']== 39114911000001105
scripts[mask_AZ].head()

,Unnamed: 0,eid,issue_date,dmd_code
4561,64814430,5521041,2021-01-20,39114911000001105
5499,16646535,2163004,2021-01-31,39114911000001105
7750,13225838,1916265,2021-02-03,39114911000001105
8932,70996867,5949279,2021-02-22,39114911000001105
9967,4432489,1304258,2021-02-02,39114911000001105


In [19]:
# getting ID's of individuals with any corona vaccine first (compare vaccinates vs. not)
# may look at effects of specific vaccines later
dict_vac = {39326911000001101: 'Moderna', 39230211000001104: 'Janssen', \
            39826711000001101:'Medicago', 39473011000001103: 'Baxter', \
            39114911000001105: 'AstraZeneca', 39115611000001103: 'Pfizer',\
            39373511000001104: 'Valneva'}

for dmd_code in dict_vac:
    mask_vac = scripts['dmd_code']== dmd_code
    print(dict_vac[dmd_code], ':',len(scripts[mask_vac]))

Moderna : 0
Janssen : 0
Medicago : 0
Baxter : 0
AstraZeneca : 23
Pfizer : 7
Valneva : 0


In [25]:
# mask for any vaccine
mask_vac = scripts['dmd_code'].isin(dict_vac)

# number of vaccinated people
print('Number of vaccinated people:', len(scripts[mask_vac]))

Number of vaccinated people: 30


In [26]:
# finding eids of vaccinated people in the dataset
id_vac = scripts[mask_vac]['eid']

In [28]:
# finding events of the vaccinated people, but only events before vaccination found 
# -> may need a more recent dataset
# need to separate clinical events before & after vaccine
mask_vac_clin = clinical['eid'].isin(id_vac)
clinical[mask_vac_clin] 

,Unnamed: 0,eid,event_dt,code_type,code,value
31073,34474547,2163004,31/12/2020,0,8HE2.,0.000000
36034,47865366,2608259,31/12/2019,0,XaIAQ,36.099998
36086,47865541,2608259,31/12/2019,0,242..,74.000000
36117,47864684,2608259,31/12/2019,0,XaJK8,0.000000
36118,47864707,2608259,31/12/2019,0,XaJKJ,0.000000
36121,47864718,2608259,31/12/2019,0,XaMiI,0.000000
36122,47864755,2608259,31/12/2019,0,XaMhk,0.000000
36126,47864891,2608259,31/12/2019,0,XaJKn,0.000000
36154,47864323,2608259,31/12/2019,0,XE0qn,0.000000
36167,47864473,2608259,31/12/2019,1,Y0529,0.000000


# categorize event as before or after COVID-19 vaccination (done by Renée)

### make 'event_date' in 'clinical' be interpreted as a date

In [47]:
clinical.head(10)

,Unnamed: 0,eid,event_dt,code_type,code,value,selected disease
0,101084316,4392480,31/12/2020,0,XaERu,11.00,NaN
1,9359934,1311969,31/12/2020,0,XaN3z,0.80,NaN
2,9360007,1311969,31/12/2020,0,44P6.,0.58,NaN
3,111269383,4731119,31/12/2020,0,XaIm8,0.00,NaN
4,9360002,1311969,31/12/2020,0,44P5.,1.30,NaN
5,111269344,4731119,31/12/2020,0,XaIge,0.00,NaN
6,9359963,1311969,31/12/2020,0,42M..,0.90,NaN
7,9359951,1311969,31/12/2020,0,XE2q4,1.11,NaN
8,9359940,1311969,31/12/2020,0,XE2eD,2.10,NaN
9,9359924,1311969,31/12/2020,0,XaELV,1.59,NaN


In [48]:
from datetime import datetime, timedelta

In [49]:
clinical['event_dt'].unique()
# find out if the other dates are e.g. 06/01/2020 or 6/1/2020

array(['31/12/2020', '31/12/2019'], dtype=object)

In [50]:
example_date = clinical['event_dt'].unique()[0]

In [51]:
example_date_new = datetime.strptime(example_date, '%d/%m/%Y')
example_date_new #2020

datetime.datetime(2020, 12, 31, 0, 0)

In [52]:
example_date_2 = clinical['event_dt'].unique()[1]
example_date_2 

'31/12/2019'

In [53]:
example_date_2_new = datetime.strptime(example_date_2, '%d/%m/%Y')
example_date_2_new #2019

datetime.datetime(2019, 12, 31, 0, 0)

In [54]:
# compare the 2 dates
example_date_new > example_date_2_new # 2020 > 2019 --> True
# recent > older --> True

True

In [55]:
# convert entire dataframe column in datetime format
clinical_copy = clinical # just to not mess up the clinical dataset
clinical_copy['event_dt_date_format'] = pd.to_datetime(clinical_copy['event_dt'])
clinical_copy

,Unnamed: 0,eid,event_dt,code_type,code,value,selected disease,event_dt_date_format
0,101084316,4392480,31/12/2020,0,XaERu,11.00,NaN,2020-12-31
1,9359934,1311969,31/12/2020,0,XaN3z,0.80,NaN,2020-12-31
2,9360007,1311969,31/12/2020,0,44P6.,0.58,NaN,2020-12-31
3,111269383,4731119,31/12/2020,0,XaIm8,0.00,NaN,2020-12-31
4,9360002,1311969,31/12/2020,0,44P5.,1.30,NaN,2020-12-31
...,...,...,...,...,...,...,...,...
49995,46497795,2564395,31/12/2019,0,XaFqt,0.00,NaN,2019-12-31
49996,46497794,2564395,31/12/2019,0,XaIVg,0.00,NaN,2019-12-31
49997,41639202,2400982,31/12/2019,0,XaEJK,0.00,NaN,2019-12-31
49998,116371391,4900832,31/12/2019,0,X1004,0.00,NaN,2019-12-31


In [56]:
#compare 2 dates (first row and last row) from the new column

print(clinical_copy['event_dt_date_format'].iloc[-1]) # 2019
print(clinical_copy['event_dt_date_format'].iloc[0]) # 2020
clinical_copy['event_dt_date_format'].iloc[0] > clinical_copy['event_dt_date_format'].iloc[-1]

2019-12-31 00:00:00
2020-12-31 00:00:00


True

### subset 'clinical' and 'scripts' to only those patients (eid) that got the COVID-19 vaccine

In [58]:
# 39114911000001105: 'AstraZeneca', 39115611000001103: 'Pfizer',
mask_PF_AZ = scripts['dmd_code'].isin([39114911000001105, 39115611000001103]) 
scripts_vaccines_only = scripts[mask_PF_AZ] 
len(scripts[mask_PF_AZ]) # corresponds to finding above: 7 pfizer and 23 astrazeneca

30

In [59]:
# extract eids
id_vac_PF_AZ = scripts_vaccines_only['eid']
id_vac_PF_AZ

# actually not needed 

4561     5521041
4849     3747009
5499     2163004
7750     1916265
8932     5949279
9967     1304258
19612    2422930
21275    2608259
23754    3113257
29233    4568234
31157    5970140
35668    1199550
37824    5251075
38147    4443306
45729    1255920
46990    2720534
47804    1326030
54582    5667376
58686    3055374
61111    4911157
65171    1295237
70822    2080101
72345    5613818
73120    5001792
80389    5245328
87512    5040390
90802    5572579
91492    4624327
96191    1560424
99070    2193600
Name: eid, dtype: int64

In [60]:
scripts_vaccines_only.head() # contains eid and date of vaccination

,Unnamed: 0,eid,issue_date,dmd_code
4561,64814430,5521041,2021-01-20,39114911000001105
4849,39329153,3747009,2021-05-02,39115611000001103
5499,16646535,2163004,2021-01-31,39114911000001105
7750,13225838,1916265,2021-02-03,39114911000001105
8932,70996867,5949279,2021-02-22,39114911000001105


In [61]:
# subset of clinical with only vaccinated 
clinical_vaccinated_only = clinical_copy[clinical_copy['eid'].isin(id_vac)]
clinical_vaccinated_only

,Unnamed: 0,eid,event_dt,code_type,code,value,selected disease,event_dt_date_format
31073,34474547,2163004,31/12/2020,0,8HE2.,0.000000,NaN,2020-12-31
36034,47865366,2608259,31/12/2019,0,XaIAQ,36.099998,NaN,2019-12-31
36086,47865541,2608259,31/12/2019,0,242..,74.000000,NaN,2019-12-31
36117,47864684,2608259,31/12/2019,0,XaJK8,0.000000,NaN,2019-12-31
36118,47864707,2608259,31/12/2019,0,XaJKJ,0.000000,NaN,2019-12-31
36121,47864718,2608259,31/12/2019,0,XaMiI,0.000000,NaN,2019-12-31
36122,47864755,2608259,31/12/2019,0,XaMhk,0.000000,NaN,2019-12-31
36126,47864891,2608259,31/12/2019,0,XaJKn,0.000000,NaN,2019-12-31
36154,47864323,2608259,31/12/2019,0,XE0qn,0.000000,NaN,2019-12-31
36167,47864473,2608259,31/12/2019,1,Y0529,0.000000,NaN,2019-12-31


### add a column with the date of vaccination to clinical_vaccinated_only

In [62]:
# attention! only run once!
# merge clinical_copy and scripts_vaccines_only to add the issue_date (date of vaccine) to the 
clinical_vac = pd.merge(scripts_vaccines_only, clinical_copy, how ='inner', on ='eid')

In [63]:
clinical_vac

,Unnamed: 0_x,eid,issue_date,dmd_code,Unnamed: 0_y,event_dt,code_type,code,value,selected disease,event_dt_date_format
0,16646535,2163004,2021-01-31,39114911000001105,34474547,31/12/2020,0,8HE2.,0.000000,NaN,2020-12-31
1,20359621,2422930,2021-01-13,39115611000001103,42290137,31/12/2019,0,X78SS,0.000000,NaN,2019-12-31
2,20359621,2422930,2021-01-13,39115611000001103,42290412,31/12/2019,0,XaIhQ,0.000000,NaN,2019-12-31
3,23019270,2608259,2021-01-22,39114911000001105,47865366,31/12/2019,0,XaIAQ,36.099998,NaN,2019-12-31
4,23019270,2608259,2021-01-22,39114911000001105,47865541,31/12/2019,0,242..,74.000000,NaN,2019-12-31
5,23019270,2608259,2021-01-22,39114911000001105,47864684,31/12/2019,0,XaJK8,0.000000,NaN,2019-12-31
6,23019270,2608259,2021-01-22,39114911000001105,47864707,31/12/2019,0,XaJKJ,0.000000,NaN,2019-12-31
7,23019270,2608259,2021-01-22,39114911000001105,47864718,31/12/2019,0,XaMiI,0.000000,NaN,2019-12-31
8,23019270,2608259,2021-01-22,39114911000001105,47864755,31/12/2019,0,XaMhk,0.000000,NaN,2019-12-31
9,23019270,2608259,2021-01-22,39114911000001105,47864891,31/12/2019,0,XaJKn,0.000000,NaN,2019-12-31


### add a column where it says if the event (event_dt_date_format) was before or after the vaccination (issue_date)

In [64]:
# is this step really needed? seems like issue_date is in date format already.
clinical_vac['issue_date_format'] = pd.to_datetime(clinical_vac['issue_date'])
clinical_vac

,Unnamed: 0_x,eid,issue_date,dmd_code,Unnamed: 0_y,event_dt,code_type,code,value,selected disease,event_dt_date_format,issue_date_format
0,16646535,2163004,2021-01-31,39114911000001105,34474547,31/12/2020,0,8HE2.,0.000000,NaN,2020-12-31,2021-01-31
1,20359621,2422930,2021-01-13,39115611000001103,42290137,31/12/2019,0,X78SS,0.000000,NaN,2019-12-31,2021-01-13
2,20359621,2422930,2021-01-13,39115611000001103,42290412,31/12/2019,0,XaIhQ,0.000000,NaN,2019-12-31,2021-01-13
3,23019270,2608259,2021-01-22,39114911000001105,47865366,31/12/2019,0,XaIAQ,36.099998,NaN,2019-12-31,2021-01-22
4,23019270,2608259,2021-01-22,39114911000001105,47865541,31/12/2019,0,242..,74.000000,NaN,2019-12-31,2021-01-22
5,23019270,2608259,2021-01-22,39114911000001105,47864684,31/12/2019,0,XaJK8,0.000000,NaN,2019-12-31,2021-01-22
6,23019270,2608259,2021-01-22,39114911000001105,47864707,31/12/2019,0,XaJKJ,0.000000,NaN,2019-12-31,2021-01-22
7,23019270,2608259,2021-01-22,39114911000001105,47864718,31/12/2019,0,XaMiI,0.000000,NaN,2019-12-31,2021-01-22
8,23019270,2608259,2021-01-22,39114911000001105,47864755,31/12/2019,0,XaMhk,0.000000,NaN,2019-12-31,2021-01-22
9,23019270,2608259,2021-01-22,39114911000001105,47864891,31/12/2019,0,XaJKn,0.000000,NaN,2019-12-31,2021-01-22


In [55]:
clinical_vac['event_dt_date_format'].iloc[0]>=clinical_vac['issue_date_format'].iloc[0]

False

In [56]:
before_after_vaccine = []
for i in range (len(clinical_vac)):
    if clinical_vac['event_dt_date_format'].iloc[i] >= clinical_vac['issue_date_format'].iloc[i]:
        before_after_vaccine = 'after'
    else:
        before_after_vaccine = 'before'
clinical_vac['before_after_vaccine'] = before_after_vaccine

In [57]:
clinical_vac

,Unnamed: 0_x,eid,issue_date,dmd_code,Unnamed: 0_y,event_dt,code_type,code,value,disease category,event_dt_date_format,issue_date_format,before_after_vaccine
0,16646535,2163004,2021-01-31,39114911000001105,34474547,31/12/2020,0,8HE2.,0.000000,NaN,2020-12-31,2021-01-31,before
1,20359621,2422930,2021-01-13,39115611000001103,42290137,31/12/2019,0,X78SS,0.000000,NaN,2019-12-31,2021-01-13,before
2,20359621,2422930,2021-01-13,39115611000001103,42290412,31/12/2019,0,XaIhQ,0.000000,NaN,2019-12-31,2021-01-13,before
3,23019270,2608259,2021-01-22,39114911000001105,47865366,31/12/2019,0,XaIAQ,36.099998,NaN,2019-12-31,2021-01-22,before
4,23019270,2608259,2021-01-22,39114911000001105,47865541,31/12/2019,0,242..,74.000000,NaN,2019-12-31,2021-01-22,before
5,23019270,2608259,2021-01-22,39114911000001105,47864684,31/12/2019,0,XaJK8,0.000000,NaN,2019-12-31,2021-01-22,before
6,23019270,2608259,2021-01-22,39114911000001105,47864707,31/12/2019,0,XaJKJ,0.000000,NaN,2019-12-31,2021-01-22,before
7,23019270,2608259,2021-01-22,39114911000001105,47864718,31/12/2019,0,XaMiI,0.000000,NaN,2019-12-31,2021-01-22,before
8,23019270,2608259,2021-01-22,39114911000001105,47864755,31/12/2019,0,XaMhk,0.000000,NaN,2019-12-31,2021-01-22,before
9,23019270,2608259,2021-01-22,39114911000001105,47864891,31/12/2019,0,XaJKn,0.000000,NaN,2019-12-31,2021-01-22,before


# Analysing clinical events

IC10 codes are structured. The following website was used for looking up the meanings of the categories (e.g. respiratory diseases):
https://www.icd10data.com/ICD10CM/Codes

In [31]:
ic10

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17931,17931,U84.8,U848,DEFAULT,3.0,Resistance to other specified antimicrobial drug,,,,,,,
17932,17932,U84.9,U849,DEFAULT,3.0,Resistance to unspecified antimicrobial drugs,,,,,,,
17933,17933,U85,U85X,DEFAULT,3.0,Resistance to antineoplastic drugs,,,,,,,
17934,17934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


the GP_ic10 has the read code which is the CTV3. 
(However, when things get nasty, there are multiple ICD10 codes per CTV3 codes and viceversa… there is a mapping status which tells you whether the code is primary, duplicated, to revision, etc)

In [30]:
# just to see if corresponding CTV3 to IC10 works

# broadcasting for vaccinated comorbidities(CTV3) in ICD10
mask_vac_ic10 = GPic10['read_code'].isin(clinical[mask_vac_clin]['code']) 
GPic10[mask_vac_ic10]

# mask_vac: people that got any vaccine
# search for patients that got the vaccine in GPic10
# and look what clinical events they had via the read_code (e.g. XE01qn)


,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
72874,72874,X78SS,C449,G,C,P,0.0,0.0
76606,76606,XE0qn,R05X,D,C,C,0.0,0.0
76607,76607,XE0qn,F453,R,C,C,0.0,0.0
76608,76608,XE0qn,J111,R,C,C,0.0,0.0
76609,76609,XE0qn,J410,R,C,C,0.0,0.0
76610,76610,XE0qn,R042,R,C,C,0.0,0.0


if time: analyse specific diseases such as myocarditis etc.

In [75]:
clinical_ic10

,Unnamed: 0,eid,event_dt,code,icd10_code
0,36821530,2238884,31/12/2020,Xabdb,R53X
1,36821530,2238884,31/12/2020,Xabdb,R54X
2,58552832,2968142,31/12/2020,Xabdb,R53X
3,58552832,2968142,31/12/2020,Xabdb,R54X
4,24993994,1832436,31/12/2020,Xabdb,R53X
...,...,...,...,...,...
4536,65073848,3185031,31/12/2019,F4201,E133D
4537,65073848,3185031,31/12/2019,F4201,O243D
4538,65073848,3185031,31/12/2019,F4201,O244D
4539,65073848,3185031,31/12/2019,F4201,O249D


In [67]:
ic10.head(3)

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,


In [74]:
# mask for getting codes related to myocarditis
mask_myocarditis = ic10['DESCRIPTION'].str.contains('myocarditis', case = False)
ic10[mask_myocarditis]

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
3700,3700,I01.2,I012,DEFAULT,3.0,Acute rheumatic myocarditis,,,,,,,
3732,3732,I09.0,I090,DEFAULT,3.0,Rheumatic myocarditis,,,,,,,
3861,3861,I40,I40,DEFAULT,3.0,Acute myocarditis,,,,,,,
3862,3862,I40.0,I400,DEFAULT,3.0,Infective myocarditis,,,,,,,
3863,3863,I40.1,I401,DEFAULT,3.0,Isolated myocarditis,,,,,,,
3864,3864,I40.8,I408,DEFAULT,3.0,Other acute myocarditis,,,,,,,
3865,3865,I40.9,I409,DEFAULT,3.0,"Acute myocarditis, unspecified",,,,,,,
3866,3866,I41,I41,ASTERIS,2.0,Myocarditis in diseases classified elsewhere,,,,,,,
3867,3867,I41.0,I410,ASTERIS,2.0,Myocarditis in bacterial diseases classified e...,,,,,,,
3868,3868,I41.1,I411,ASTERIS,2.0,Myocarditis in viral diseases classified elsew...,,,,,,,


In [77]:
# getting data with myocarditis

# the alternative code 'ALT_CODE' used here, as this was used in the lookup table GPic10
myocarditis = ic10[mask_myocarditis]['ALT_CODE']

clinical_ic10[clinical_ic10['icd10_code'].isin(myocarditis)]
# nobody had myocarditis in this restricted data

,Unnamed: 0,eid,event_dt,code,icd10_code


In [78]:
## ishaemic heart disease
mask_ihd = ic10['DESCRIPTION'].str.contains('ischaemic heart disease', case = False)
ischaemicHD = ic10[mask_ihd]['ALT_CODE']
clinical_ic10[clinical_ic10['icd10_code'].isin(ischaemicHD)]

,Unnamed: 0,eid,event_dt,code,icd10_code
1018,83141410,3791841,31/12/2020,XE2uV,I259
1019,83141410,3791841,31/12/2020,XE2uV,I25
1020,83141410,3791841,31/12/2020,XE2uV,I24
1021,110210389,4695920,31/12/2020,XE2uV,I259
1022,110210389,4695920,31/12/2020,XE2uV,I25
1023,110210389,4695920,31/12/2020,XE2uV,I24
1024,25621154,1854201,31/12/2020,XE2uV,I259
1025,25621154,1854201,31/12/2020,XE2uV,I25
1026,25621154,1854201,31/12/2020,XE2uV,I24
1027,30444676,2022763,31/12/2019,XE2uV,I259


several comorbidities

In [79]:
# getting people with specific diseases

# new column for filling with selected disease
clinical_ic10['selected disease'] = np.nan 

# list of specific diseases, to be decided on
diseases_selec = ['ischaemic heart disease', 'cardiomyopathy', 'heart failure',\
                  'myocarditis'] 

for disease in diseases_selec:
    
    # broadcasting for selected disease
    mask = ic10['DESCRIPTION'].str.contains(disease, case = False) 
    # broadcasting for selected disease
    sub_disease = clinical_ic10[clinical_ic10['icd10_code'].isin(ic10[mask]['ALT_CODE'])]
    
    for i in sub_disease.index:
        # fill dataset with selected disease label
        clinical_ic10.loc[i,'selected disease'] = disease 

In [80]:
# counting how many people in the dataset have a specific disease
clinical_ic10['selected disease'].value_counts()

ischaemic heart disease    31
heart failure              16
cardiomyopathy              2
Name: selected disease, dtype: int64

# code layout for getting categories of people w. different comorbidities

In [34]:
# code layout for getting the different categories

# subsetting into categories using IC10
# e.g. I00-I99 Diseases of the circulatory system, more details available too, e.g. Ischemic heart diseases
GPic10.dropna(inplace=True) # drop NaN to enable making mask

m_circ = GPic10['icd10_code'].str.startswith('I')
GPic10[m_circ]['read_code']

302       14A3.
303       14A4.
600       182..
602       1822.
610       1828.
          ...  
115198    Xafis
115199    Xafis
115200    Xafis
115204    Xafyq
115207    Xafyq
Name: read_code, Length: 3771, dtype: object

In [35]:
# getting corresponding CTV3 codes
m_circ_CTV3 = code['coding'].isin(GPic10[m_circ]['read_code'])
code[m_circ_CTV3]

,coding,meaning
34065,14A3.,H/O: myocardial infarct at less than 60
34066,14A4.,H/O: myocardial infarct at greater than 60
34881,182..,Chest pain
34883,1822.,Central chest pain
34889,1828.,Atypical chest pain
...,...,...
180522,XM1Qq,Thrombophlebitis
180523,XM1Qr,Superficial vein thrombosis
180524,XM1Qs,Perineal varices
180526,XM1Qu,Claudication


In [36]:
 # getting subjects with this condition
m_circ_sub = clinical['code'].isin(code[m_circ_CTV3]['coding']) # another mask

# correspond to vaccinated or not

In [38]:
# no one with any vaccine had circulatory issue before vaccine on this limited dataset
scripts[mask_vac]['eid'].isin(clinical[m_circ_sub]['eid']).sum()

0